In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\n  "intents": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n\t  "Hi Bot",\n\t  "Hola",\n\t  "Hi, how are you",\n        "How are you",\n        "Is anyone there?",\n        "Hello",\n        "Hey",\n        "What\'s up",\n        "Hi there",\n        "Hey there",\n        "Good morning",\n        "Good afternoon",\n        "Good evening",\n        "Howdy",\n        "Hello there"\n      ],\n      "responses": [\n        "What can i help you with today?",\n        "How can I help you today?"\n      ],\n      "context_set": ""\n    },\n    {\n      "tag": "goodbye",\n      "patterns": [\n        "Bye",\n        "See you later",\n        "Goodbye",\n        "Farewell",\n        "Take care",\n        "Catch you later",\n        "Goodbye for now",\n        "Bye for now",\n        "Talk to you later",\n        "See you soon",\n        "Take it easy",\n        "So long",\n        "Adios",\n        "Good night",\n        "Later",\n        "Have a g

In [ ]:
import gensim.downloader as api
# Load pre-trained word2vec model
model = api.load("word2vec-google-news-300")

[=================================================-] 99.7% 1658.1/1662.8MB downloaded


In [ ]:
import gensim.downloader as api
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from tensorflow import keras
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Load the intents data
with open("intents.json", "r") as f:
    data = json.load(f)

# Preprocessing
vocab = set()
classes = []
documents = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern = pattern.rstrip("?")
        tokens = word_tokenize(pattern)
        tokens = [token.lower() for token in tokens if token not in set(stopwords.words("english"))]
        vocab.update(tokens)
        documents.append((tokens, intent["tag"]))
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Generate word embeddings for each token
X = []
y = []
for doc in documents:
    vecs = []
    for token in doc[0]:
        if token in model.key_to_index:
            vecs.append(model[token])
    if vecs:
        vecs = np.mean(vecs, axis=0)
        X.append(vecs)
        y.append(classes.index(doc[1]))

X = np.array(X)
y = np.array(y)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Reshape the data for RNN
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


# Build the model
model1 = Sequential()
model1.add(SimpleRNN(128, input_shape=(X_train.shape[1], X_train.shape[2]), activation="relu", return_sequences=True))
model1.add(Dropout(0.5))
model1.add(SimpleRNN(64, activation="relu"))
model1.add(Dense(len(classes), activation="softmax"))

model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto', restore_best_weights=True)

# Train the model
model1.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=5, callbacks=early_stop)

# Evaluate the model
loss, accuracy = model1.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
def vectorize_sentence(sentence, model):
    sentence = sentence.rstrip("?")
    # tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence)
    # lemmatize each word and remove stopwords
    sentence_words = [lemmatizer.lemmatize(word.lower(), pos='v') for word in sentence_words if word not in set(stopwords.words("english"))]
    # create a vector representation
    vecs = []
    for word in sentence_words:
        if word in model.key_to_index:
            vecs.append(model[word])
    if vecs:
        vecs = np.mean(vecs, axis=0)
    else:
        vecs = np.zeros(model.vector_size)
    return vecs

print("Hi there, this is Genie. I'm so happy to chat with you today! 😊 \nCould you please be more specific with your questions.\nI'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):")
while True:
    inp = input("You: ")
    if inp.lower() == "quit":
        for intent in data["intents"]:
            if intent["tag"] == "goodbye":
                print("Bot: " + random.choice(intent["responses"]))
        break
    vec = vectorize_sentence(inp, model)
    vec = np.expand_dims(vec, axis=0)  # Add an extra dimension for batch
    vec = vec.reshape(1, 1, vec.shape[1])  # Reshape to match the input shape of the model
    results = model1.predict(vec)
    results_index = np.argmax(results)
    tag = classes[results_index]
    if results[0][results_index] > 0.5:
        for intent in data["intents"]:
            if intent["tag"] == tag:
                if intent["tag"] == "goodbye" and inp == 'quit':
                    print("Bot: " + random.choice(intent["responses"]))
                    print("Bot: Goodbye!")
                    break
                if intent["tag"] == "goodbye" and inp != 'quit':
                    print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist")
                else:
                    print("Bot: " + random.choice(intent["responses"]))
                    break
    else:
        print("Bot: I am evolving constantly. I apologize that I could not help you with your query. Please give us a call at  and our team will be happy to assist")


Hi there, this is Genie. I'm so happy to chat with you today! 😊 
Could you please be more specific with your questions.
I'm always trying to improve my communication skills and learn from your feedback. (type quit to end the conversation):
You: How much time will it take for the order to be prepared?
1/1 [==============================] - 3s 3s/step
Bot: We have many great options to choose from, You can place your order online at https://www.suggioota.com/order-online
You: I need to order for catering.
1/1 [==============================] - 0s 22ms/step
Bot: We'd love to help make your event unforgettable with our fresh and delicious food. Feel free to give us a call at (669)-224-0833 to discuss your catering needs.
You: Is home Delivery available or pickup?
1/1 [==============================] - 0s 19ms/step
Bot: We have home delivery options through Doordash, Uber Eats, Grubhub, and LemonHat
You: what are the vegetarian options?
1/1 [==============================] - 0s 20ms/step
Bo

In [ ]:
path='/content/drive/My Drive/Colab Notebooks' #Declare the path where the code and model file has been placed

In [ ]:
model1.save('Genie_RNN.h5')#save the model